# DATA PIPELINE

In [32]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np 
import joblib
import os
import yaml

In [33]:
!ls

config
data
ml_process_2_4_data_pipelines.ipynb
ml_process_3_5_EDA.ipynb


In [34]:
!cat config/params.yaml

dataset_dir: data/raw/
datetime_columns:
- tanggal
int32_columns:
- pm10
- pm25
- so2
- co
- o3
- no2
- max
label: categori
label_categories:
- BAIK
- SEDANG
- TIDAK SEHAT
label_categories_new:
- BAIK
- TIDAK BAIK
missing_value_co: 11
missing_value_no2: 18
missing_value_o3: 29
missing_value_pm10:
  BAIK: 28
  TIDAK BAIK: 55
missing_value_pm25:
  BAIK: 38
  TIDAK BAIK: 82
missing_value_so2: 35
object_columns:
- stasiun
- critical
- categori
predictors:
- stasiun
- pm10
- pm25
- so2
- co
- o3
- no2
range_co:
- -1
- 100
range_no2:
- -1
- 100
range_o3:
- -1
- 160
range_pm10:
- -1
- 800
range_pm25:
- -1
- 400
range_so2:
- -1
- 500
range_stasiun:
- DKI1 (Bunderan HI)
- DKI2 (Kelapa Gading)
- DKI3 (Jagakarsa)
- DKI4 (Lubang Buaya)
- DKI5 (Kebon Jeruk) Jakarta Barat


In [35]:
params_dir = "config/params.yaml"

In [36]:
def load_params(param_dir):
    with open(param_dir, 'r') as file:
        params = yaml.safe_load(file)
        
    return params

In [37]:
params = load_params(params_dir)

In [38]:
params

{'dataset_dir': 'data/raw/',
 'datetime_columns': ['tanggal'],
 'int32_columns': ['pm10', 'pm25', 'so2', 'co', 'o3', 'no2', 'max'],
 'label': 'categori',
 'label_categories': ['BAIK', 'SEDANG', 'TIDAK SEHAT'],
 'label_categories_new': ['BAIK', 'TIDAK BAIK'],
 'missing_value_co': 11,
 'missing_value_no2': 18,
 'missing_value_o3': 29,
 'missing_value_pm10': {'BAIK': 28, 'TIDAK BAIK': 55},
 'missing_value_pm25': {'BAIK': 38, 'TIDAK BAIK': 82},
 'missing_value_so2': 35,
 'object_columns': ['stasiun', 'critical', 'categori'],
 'predictors': ['stasiun', 'pm10', 'pm25', 'so2', 'co', 'o3', 'no2'],
 'range_co': [-1, 100],
 'range_no2': [-1, 100],
 'range_o3': [-1, 160],
 'range_pm10': [-1, 800],
 'range_pm25': [-1, 400],
 'range_so2': [-1, 500],
 'range_stasiun': ['DKI1 (Bunderan HI)',
  'DKI2 (Kelapa Gading)',
  'DKI3 (Jagakarsa)',
  'DKI4 (Lubang Buaya)',
  'DKI5 (Kebon Jeruk) Jakarta Barat']}

## 1. Data Collection

In [39]:
# fungsi untuk membaca nama file, memuat file, dan menggabungkan dataset
def read_dataset(dataset_dir):
    dataset = pd.DataFrame()

    for i in tqdm(os.listdir(dataset_dir)):
        dataset = pd.concat([pd.read_csv(dataset_dir + i), dataset])
    
    return dataset

In [40]:
# melakukan pembacaan nama file, memuat file, dan menggabungkan dataset
dataset = read_dataset(params["dataset_dir"])

100%|██████████| 12/12 [00:00<00:00, 323.25it/s]


In [41]:
dataset

,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,critical,categori
0,2021-09-01,DKI1 (Bunderan HI),63,88,29,15,24,38,88,PM25,SEDANG
1,2021-09-02,DKI1 (Bunderan HI),60,83,29,11,30,28,83,PM25,SEDANG
2,2021-09-03,DKI1 (Bunderan HI),60,82,27,11,37,30,82,PM25,SEDANG
3,2021-09-04,DKI1 (Bunderan HI),58,77,26,10,31,28,77,PM25,SEDANG
4,2021-09-05,DKI1 (Bunderan HI),63,85,27,11,28,28,85,PM25,SEDANG
...,...,...,...,...,...,...,...,...,...,...,...
150,2021-08-27,DKI5 (Kebon Jeruk) Jakarta Barat,61,96,34,8,29,15,96,PM25,SEDANG
151,2021-08-28,DKI5 (Kebon Jeruk) Jakarta Barat,63,100,31,8,44,12,100,PM25,SEDANG
152,2021-08-29,DKI5 (Kebon Jeruk) Jakarta Barat,67,111,32,10,36,13,111,PM25,TIDAK SEHAT
153,2021-08-30,DKI5 (Kebon Jeruk) Jakarta Barat,83,126,35,16,32,29,126,PM25,TIDAK SEHAT


In [42]:
# terdapat beberapa temuan disini:
# 1. index hanya terlihat sampai 154 padahal jumlah rows sampai 1070
# 2. tanggal hanya terlihat dari bulan 8 dan bulan 9

# harus diselidiki lebih lanjut
# reset index untuk mengatasi poin pertama
dataset.reset_index(inplace = True, drop = True)

In [43]:
# lakukan pengecekan hasil
dataset

,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,critical,categori
0,2021-09-01,DKI1 (Bunderan HI),63,88,29,15,24,38,88,PM25,SEDANG
1,2021-09-02,DKI1 (Bunderan HI),60,83,29,11,30,28,83,PM25,SEDANG
2,2021-09-03,DKI1 (Bunderan HI),60,82,27,11,37,30,82,PM25,SEDANG
3,2021-09-04,DKI1 (Bunderan HI),58,77,26,10,31,28,77,PM25,SEDANG
4,2021-09-05,DKI1 (Bunderan HI),63,85,27,11,28,28,85,PM25,SEDANG
...,...,...,...,...,...,...,...,...,...,...,...
1820,2021-08-27,DKI5 (Kebon Jeruk) Jakarta Barat,61,96,34,8,29,15,96,PM25,SEDANG
1821,2021-08-28,DKI5 (Kebon Jeruk) Jakarta Barat,63,100,31,8,44,12,100,PM25,SEDANG
1822,2021-08-29,DKI5 (Kebon Jeruk) Jakarta Barat,67,111,32,10,36,13,111,PM25,TIDAK SEHAT
1823,2021-08-30,DKI5 (Kebon Jeruk) Jakarta Barat,83,126,35,16,32,29,126,PM25,TIDAK SEHAT


In [44]:
# simpan dataset yang telah digabungkan
joblib.dump(dataset, "data/processed/dataset.pkl")

['data/processed/dataset.pkl']

In [45]:
! ls data/processed

dataset.pkl
dataset_clean.pkl
x_test.pkl
x_train.pkl
x_valid.pkl
y_test.pkl
y_train.pkl
y_valid.pkl


## 2. Data Definition

In [46]:
# definisikan tipe data, range data serta penjelasan untuk tiap observasi (variabel)

tanggal         :
    [datetime]
    [00:00 01/06/2021 - 23:59 31/12/2021]
    waktu saat pengambilan sampel

stasiun         :
    [object]
    ['DKI1 (Bunderan HI)', 'DKI2 (Kelapa Gading)', 'DKI3 (Jagakarsa)', 'DKI4 (Lubang Buaya)', 'DKI5 (Kebon Jeruk) Jakarta Barat']
    lokasi saat pengambilan sampel

pm10            :
    [integer]
    [0 - 800]
    partikel udara yang berukuran lebih kecil dari 10 mikron

pm25            :
    [integer]
    [0 - 400]
    partikel udara yang berukuran lebih kecil dari 2.5 mikron

so2             :
    [integer]
    [0 - 500]
    sulfur dioksida

co              :
    [integer]
    [0 - 100]
    karbon monoksida

o3              :
    [integer]
    [0 - 140]
    ozone

no2             :
    [integer]
    [0 - 100]
    nitrogen dioksida

max             :
    [integer]
    [0 - 800]
    nilai paling besar diantara pm10, pm25, so2, co, o3, dan no2

critical        :
    [object]
    [PM10, PM25, SO2, CO, O3, dan NO2]
    nama kolom untuk nilai max

categori        :
    [object]
    [BAIK, SEDANG, TIDAK SEHAT]
    kategori untuk data pengukuran udara

location        : tidak termasuk karena ada data yang nama kolomnya tidak standar

## 3. Data Validation

In [47]:
# cek tipe data
dataset.dtypes

tanggal     object
stasiun     object
pm10        object
pm25        object
so2         object
co          object
o3          object
no2         object
max         object
critical    object
categori    object
dtype: object

In [48]:
# dari pengecekan data terlihat bahwa semuanya adalah data objek (string), perlu diselidiki lebih lanjut

In [49]:
# pengecekan cakupan data menjadi kacau jika tipe data tidak sesuai
dataset.describe()

,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,critical,categori
count,1825,1825,1825,1763,1825,1825,1825,1825,1825,1809,1824
unique,365,5,130,192,121,54,121,96,187,6,4
top,2021-09-01,DKI1 (Bunderan HI),51,77,---,9,---,13,77,PM25,SEDANG
freq,5,365,69,45,114,163,68,81,50,1630,1349


In [50]:
# dimensi data kemungkinan besar tidak terpengaruh, namun nanti kita kembali lagi
dataset.shape

(1825, 11)

In [51]:
# untuk handling column error, cek column yg error

In [52]:
# cek tipe data pada kolom tanggal
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1825 entries, 0 to 1824
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tanggal   1825 non-null   object
 1   stasiun   1825 non-null   object
 2   pm10      1825 non-null   object
 3   pm25      1763 non-null   object
 4   so2       1825 non-null   object
 5   co        1825 non-null   object
 6   o3        1825 non-null   object
 7   no2       1825 non-null   object
 8   max       1825 non-null   object
 9   critical  1809 non-null   object
 10  categori  1824 non-null   object
dtypes: object(11)
memory usage: 157.0+ KB


In [53]:
# casting tipe data ke datetime
dataset.tanggal = pd.to_datetime(dataset.tanggal)

In [54]:
# cek perubahan tipe data untuk kolom tanggal
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1825 entries, 0 to 1824
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1825 non-null   datetime64[ns]
 1   stasiun   1825 non-null   object        
 2   pm10      1825 non-null   object        
 3   pm25      1763 non-null   object        
 4   so2       1825 non-null   object        
 5   co        1825 non-null   object        
 6   o3        1825 non-null   object        
 7   no2       1825 non-null   object        
 8   max       1825 non-null   object        
 9   critical  1809 non-null   object        
 10  categori  1824 non-null   object        
dtypes: datetime64[ns](1), object(10)
memory usage: 157.0+ KB


In [55]:
# terlihat tidak ada masalah pada kolom pm10
dataset

,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,critical,categori
0,2021-09-01,DKI1 (Bunderan HI),63,88,29,15,24,38,88,PM25,SEDANG
1,2021-09-02,DKI1 (Bunderan HI),60,83,29,11,30,28,83,PM25,SEDANG
2,2021-09-03,DKI1 (Bunderan HI),60,82,27,11,37,30,82,PM25,SEDANG
3,2021-09-04,DKI1 (Bunderan HI),58,77,26,10,31,28,77,PM25,SEDANG
4,2021-09-05,DKI1 (Bunderan HI),63,85,27,11,28,28,85,PM25,SEDANG
...,...,...,...,...,...,...,...,...,...,...,...
1820,2021-08-27,DKI5 (Kebon Jeruk) Jakarta Barat,61,96,34,8,29,15,96,PM25,SEDANG
1821,2021-08-28,DKI5 (Kebon Jeruk) Jakarta Barat,63,100,31,8,44,12,100,PM25,SEDANG
1822,2021-08-29,DKI5 (Kebon Jeruk) Jakarta Barat,67,111,32,10,36,13,111,PM25,TIDAK SEHAT
1823,2021-08-30,DKI5 (Kebon Jeruk) Jakarta Barat,83,126,35,16,32,29,126,PM25,TIDAK SEHAT


In [56]:
# namun kolom pm10 bukanlah bertipe integer
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1825 entries, 0 to 1824
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1825 non-null   datetime64[ns]
 1   stasiun   1825 non-null   object        
 2   pm10      1825 non-null   object        
 3   pm25      1763 non-null   object        
 4   so2       1825 non-null   object        
 5   co        1825 non-null   object        
 6   o3        1825 non-null   object        
 7   no2       1825 non-null   object        
 8   max       1825 non-null   object        
 9   critical  1809 non-null   object        
 10  categori  1824 non-null   object        
dtypes: datetime64[ns](1), object(10)
memory usage: 157.0+ KB


In [57]:
# coba langsung casting ke integer
dataset.pm10 = dataset.pm10.astype(int)

ValueError: invalid literal for int() with base 10: '---'

In [59]:
# ternyata ada kategori yang bukan numerik teks, yaitu '---', maka dari itu castingnya gagal
# kita akan melakukan replacing ke data '---' dengan suatu angka numerik, maka dari itu kita harus mencari angka yang unik untuk merepresentasikannya

# angka -1 merupakan nilai yang bisa digunakan karena tidak terdapat di semua kolom dalam dataset kita
# nilai pengganti tidak harus -1, bisa apapun yang penting dapat dengan mudah di bedakan dan unik
dataset[(dataset.eq("-1").any(1)) | (dataset.eq(-1).any(1))]

C:\Users\Opal\AppData\Local\Temp\ipykernel_17556\2091607993.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  dataset[(dataset.eq("-1").any(1)) | (dataset.eq(-1).any(1))]


,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,critical,categori
1163,2021-01-01,DKI4 (Lubang Buaya),41,-1,37,14,35,4,41,PM10,BAIK
1164,2021-01-02,DKI4 (Lubang Buaya),44,-1,38,38,59,3,59,CO,SEDANG
1165,2021-01-03,DKI4 (Lubang Buaya),50,-1,37,16,52,5,52,CO,SEDANG
1166,2021-01-04,DKI4 (Lubang Buaya),45,-1,36,16,24,3,45,PM10,BAIK
1167,2021-01-05,DKI4 (Lubang Buaya),52,-1,36,15,35,8,52,PM10,SEDANG
...,...,...,...,...,...,...,...,...,...,...,...
1220,2021-01-27,DKI5 (Kebon Jeruk) Jakarta Barat,15,-1,16,5,20,---,20,CO,BAIK
1221,2021-01-28,DKI5 (Kebon Jeruk) Jakarta Barat,28,-1,19,11,28,---,28,PM10,BAIK
1222,2021-01-29,DKI5 (Kebon Jeruk) Jakarta Barat,30,-1,22,10,32,11,32,CO,BAIK
1223,2021-01-30,DKI5 (Kebon Jeruk) Jakarta Barat,24,-1,17,8,27,10,27,CO,BAIK


In [60]:
# replace data teks dengan -1
# kita tidak bisa replace dengan NaN karena NaN tidak bisa di casting ke integer
dataset.pm10 = dataset.pm10.replace(["---"], -1).astype(int)

In [61]:
# cek tipe data
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1825 entries, 0 to 1824
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1825 non-null   datetime64[ns]
 1   stasiun   1825 non-null   object        
 2   pm10      1825 non-null   int32         
 3   pm25      1825 non-null   object        
 4   so2       1825 non-null   object        
 5   co        1825 non-null   object        
 6   o3        1825 non-null   object        
 7   no2       1825 non-null   object        
 8   max       1825 non-null   object        
 9   critical  1809 non-null   object        
 10  categori  1824 non-null   object        
dtypes: datetime64[ns](1), int32(1), object(9)
memory usage: 149.8+ KB


In [62]:
# cek kolom pm25
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1825 entries, 0 to 1824
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1825 non-null   datetime64[ns]
 1   stasiun   1825 non-null   object        
 2   pm10      1825 non-null   int32         
 3   pm25      1825 non-null   object        
 4   so2       1825 non-null   object        
 5   co        1825 non-null   object        
 6   o3        1825 non-null   object        
 7   no2       1825 non-null   object        
 8   max       1825 non-null   object        
 9   critical  1809 non-null   object        
 10  categori  1824 non-null   object        
dtypes: datetime64[ns](1), int32(1), object(9)
memory usage: 149.8+ KB


In [63]:
# coba langsung casting ke integer
dataset.pm25 = dataset.pm25.astype(int)

ValueError: invalid literal for int() with base 10: '---'

In [64]:
# juga tidak berhasil selayaknya pm10, ada data yang bukan alphanumerik yaitu '---' sehingga gagal melakukan casting
# terdapat NaN, maka dari itu untuk sementara replace terlebih dahulu dengan -1 seperti data '---'
dataset.pm25.isna().sum()

0

In [65]:
# replacing NaN dengan -1
dataset.pm25.fillna(-1, inplace = True)

In [66]:
# coba lagi untuk casting ke integer
dataset.pm25 = dataset.pm25.replace("---", -1).astype(int)

In [67]:
# cek tipe data
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1825 entries, 0 to 1824
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1825 non-null   datetime64[ns]
 1   stasiun   1825 non-null   object        
 2   pm10      1825 non-null   int32         
 3   pm25      1825 non-null   int32         
 4   so2       1825 non-null   object        
 5   co        1825 non-null   object        
 6   o3        1825 non-null   object        
 7   no2       1825 non-null   object        
 8   max       1825 non-null   object        
 9   critical  1809 non-null   object        
 10  categori  1824 non-null   object        
dtypes: datetime64[ns](1), int32(2), object(8)
memory usage: 142.7+ KB


In [ ]:
# semua yang dilakukan pada pm10 dan pm25 juga diterapkan pada tiap kolom yang seharusnya mempunyai data integer

In [68]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1825 entries, 0 to 1824
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1825 non-null   datetime64[ns]
 1   stasiun   1825 non-null   object        
 2   pm10      1825 non-null   int32         
 3   pm25      1825 non-null   int32         
 4   so2       1825 non-null   object        
 5   co        1825 non-null   object        
 6   o3        1825 non-null   object        
 7   no2       1825 non-null   object        
 8   max       1825 non-null   object        
 9   critical  1809 non-null   object        
 10  categori  1824 non-null   object        
dtypes: datetime64[ns](1), int32(2), object(8)
memory usage: 142.7+ KB


In [69]:
dataset.so2 = dataset.so2.astype(int)

ValueError: invalid literal for int() with base 10: '---'

In [70]:
dataset.so2 = dataset.so2.replace("---", -1).astype(int)

In [71]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1825 entries, 0 to 1824
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1825 non-null   datetime64[ns]
 1   stasiun   1825 non-null   object        
 2   pm10      1825 non-null   int32         
 3   pm25      1825 non-null   int32         
 4   so2       1825 non-null   int32         
 5   co        1825 non-null   object        
 6   o3        1825 non-null   object        
 7   no2       1825 non-null   object        
 8   max       1825 non-null   object        
 9   critical  1809 non-null   object        
 10  categori  1824 non-null   object        
dtypes: datetime64[ns](1), int32(3), object(7)
memory usage: 135.6+ KB


In [72]:
dataset.co = dataset.co.astype(int)

ValueError: invalid literal for int() with base 10: '---'

In [73]:
dataset.co = dataset.co.replace("---", -1).astype(int)

In [74]:
dataset.o3 = dataset.o3.astype(int)

ValueError: invalid literal for int() with base 10: '---'

In [75]:
dataset.o3 = dataset.o3.replace("---", -1).astype(int)

In [76]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1825 entries, 0 to 1824
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1825 non-null   datetime64[ns]
 1   stasiun   1825 non-null   object        
 2   pm10      1825 non-null   int32         
 3   pm25      1825 non-null   int32         
 4   so2       1825 non-null   int32         
 5   co        1825 non-null   int32         
 6   o3        1825 non-null   int32         
 7   no2       1825 non-null   object        
 8   max       1825 non-null   object        
 9   critical  1809 non-null   object        
 10  categori  1824 non-null   object        
dtypes: datetime64[ns](1), int32(5), object(5)
memory usage: 121.3+ KB


In [77]:
dataset.no2 = dataset.no2.replace("---", -1).astype(int)

In [78]:
dataset["max"] = dataset["max"].astype(int)

ValueError: invalid literal for int() with base 10: 'PM25'

In [79]:
# untuk kegagalan kali ini disebabkan oleh data yang berbeda dengan sebelumnya, yaitu "PM25" instead of "---"
# perlu kita perhatikan lebih detail
dataset[dataset["max"] == "PM25"]

,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,critical,categori
1367,2021-12-03,DKI1 (Bunderan HI),49,31,9,19,7,49,PM25,BAIK,NaN


In [80]:
# dari row 1367, dapat kita simpulkan bahwa data kolom max bisa dari pm10 atau no2
# data pada kolom critical adalah PM25, namun itu tidak sesuai dengan definisinya
# maka dari itu kita akan memilih antara PM10 atau NO2
# dan kolom categori, kita dapat berasumsi bahwa nilainya adalah BAIK
# namun dengan salahnya data pada kolom critical (data PM25 namun bukan PM25 yang kritikal, melaikan PM10 dan NO2) maka asumsi kitapun bisa salah
# untuk saat ini kita set dulu ke BAIK untuk nilai di kolom categori, nanti kita liat polanya di EDA untuk categori BAIK
# ingat-ingat saja indexnya adalah 1367

# quick fix the problem
dataset.loc[1367, "max"] = 49
dataset.loc[1367, "critical"] = "PM10"
dataset.loc[1367, "categori"] = "BAIK"

In [81]:
# cek ulang hasilnya
dataset[dataset["max"] == "PM25"]

,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,critical,categori


In [82]:
dataset.loc[1367]

tanggal     2021-12-03 00:00:00
stasiun      DKI1 (Bunderan HI)
pm10                         49
pm25                         31
so2                           9
co                           19
o3                            7
no2                          49
max                          49
critical                   PM10
categori                   BAIK
Name: 1367, dtype: object

In [83]:
dataset["max"] = dataset["max"].astype(int)

In [84]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1825 entries, 0 to 1824
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1825 non-null   datetime64[ns]
 1   stasiun   1825 non-null   object        
 2   pm10      1825 non-null   int32         
 3   pm25      1825 non-null   int32         
 4   so2       1825 non-null   int32         
 5   co        1825 non-null   int32         
 6   o3        1825 non-null   int32         
 7   no2       1825 non-null   int32         
 8   max       1825 non-null   int32         
 9   critical  1809 non-null   object        
 10  categori  1825 non-null   object        
dtypes: datetime64[ns](1), int32(7), object(3)
memory usage: 107.1+ KB


In [85]:
# cek data unik pada kolom kategorik "critical"
dataset.critical.value_counts()

PM25    1630
PM10      64
O3        56
CO        34
SO2       25
Name: critical, dtype: int64

In [ ]:
# semuanya terlihat normal, tidak diperlukan perlakuan khusus

In [86]:
# cek data unik untuk kolom kategorik "categori" yang merupakan data label atau dependen variabel
dataset.categori.value_counts()

SEDANG            1349
TIDAK SEHAT        272
BAIK               188
TIDAK ADA DATA      16
Name: categori, dtype: int64

In [87]:
# terdapat data "TIDAK ADA DATA" yang mengindikasikan null value
# bisa kita langsung hapus (drop)

dataset.drop(index = dataset[dataset.categori == "TIDAK ADA DATA"].index, inplace = True)

In [88]:
dataset.categori.value_counts()

SEDANG         1349
TIDAK SEHAT     272
BAIK            188
Name: categori, dtype: int64

In [89]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1809 entries, 0 to 1824
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1809 non-null   datetime64[ns]
 1   stasiun   1809 non-null   object        
 2   pm10      1809 non-null   int32         
 3   pm25      1809 non-null   int32         
 4   so2       1809 non-null   int32         
 5   co        1809 non-null   int32         
 6   o3        1809 non-null   int32         
 7   no2       1809 non-null   int32         
 8   max       1809 non-null   int32         
 9   critical  1809 non-null   object        
 10  categori  1809 non-null   object        
dtypes: datetime64[ns](1), int32(7), object(3)
memory usage: 120.1+ KB


In [90]:
dataset.categori.value_counts()

SEDANG         1349
TIDAK SEHAT     272
BAIK            188
Name: categori, dtype: int64

In [ ]:
# semua tipe data sudah sesuai, bisa kita simpan agar nanti dapat digunakan kembali

In [91]:
joblib.dump(dataset, "data/processed/dataset_clean.pkl")

['data/processed/dataset_clean.pkl']

## 4. Data Defense

In [92]:
def check_data(input_data, params):
    # check data types
    assert input_data.select_dtypes("datetime").columns.to_list() == params["datetime_columns"], "an error occurs in datetime column(s)."
    assert input_data.select_dtypes("object").columns.to_list() == params["object_columns"], "an error occurs in object column(s)."
    assert input_data.select_dtypes("int").columns.to_list() == params["int32_columns"], "an error occurs in int32 column(s)."

    # check range of data
    assert set(input_data.stasiun).issubset(set(params["range_stasiun"])), "an error occurs in stasiun range."
    assert input_data.pm10.between(params["range_pm10"][0], params["range_pm10"][1]).sum() == len(input_data), "an error occurs in pm10 range."
    assert input_data.pm25.between(params["range_pm25"][0], params["range_pm25"][1]).sum() == len(input_data), "an error occurs in pm25 range."
    assert input_data.so2.between(params["range_so2"][0], params["range_so2"][1]).sum() == len(input_data), "an error occurs in so2 range."
    assert input_data.co.between(params["range_co"][0], params["range_co"][1]).sum() == len(input_data), "an error occurs in co range."
    assert input_data.o3.between(params["range_o3"][0], params["range_o3"][1]).sum() == len(input_data), "an error occurs in o3 range."
    assert input_data.no2.between(params["range_no2"][0], params["range_no2"][1]).sum() == len(input_data), "an error occurs in no2 range."

In [93]:
# jika tidak ada error berarti data sudah sesuai dengan desain kita
check_data(dataset, params)

## 5. Data Splitting

In [94]:
# pisahkan data x dan y (x adalah fitur, y adalah label)
x = dataset[params["predictors"]].copy()
y = dataset.categori.copy()

In [ ]:
# dataset x adalah untuk input data
# dataset y adalah output yang dihasilkan (berupa kategori: baik, sedang, dan buruk)

In [95]:
print(params["predictors"])

['stasiun', 'pm10', 'pm25', 'so2', 'co', 'o3', 'no2']


In [96]:
print(dataset.categori.unique())

['SEDANG' 'BAIK' 'TIDAK SEHAT']


In [97]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1809 entries, 0 to 1824
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   stasiun  1809 non-null   object
 1   pm10     1809 non-null   int32 
 2   pm25     1809 non-null   int32 
 3   so2      1809 non-null   int32 
 4   co       1809 non-null   int32 
 5   o3       1809 non-null   int32 
 6   no2      1809 non-null   int32 
dtypes: int32(6), object(1)
memory usage: 70.7+ KB


In [98]:
y.value_counts()

SEDANG         1349
TIDAK SEHAT     272
BAIK            188
Name: categori, dtype: int64

In [99]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42, stratify = y)
# random state merupakan variable yg digunakan untuk randomisasi thd data, supaya randomisasi ini identik dan bisa digunakan kembali di kasus berikutnya
# stratify merupakan parameter yang digunakan untuk mengetahui y manakah yg akan dipakai

In [100]:
x_valid, x_test, y_valid, y_test = train_test_split(x_test, y_test, test_size = 0.5, random_state = 42, stratify = y_test)

In [101]:
joblib.dump(x_train, "data/processed/x_train.pkl")
joblib.dump(y_train, "data/processed/y_train.pkl")
joblib.dump(x_valid, "data/processed/x_valid.pkl")
joblib.dump(y_valid, "data/processed/y_valid.pkl")
joblib.dump(x_test, "data/processed/x_test.pkl")
joblib.dump(y_test, "data/processed/y_test.pkl")

['data/processed/y_test.pkl']

In [102]:
!ls data/processed/

dataset.pkl
dataset_clean.pkl
x_test.pkl
x_train.pkl
x_valid.pkl
y_test.pkl
y_train.pkl
y_valid.pkl
